# Análisis de Datos - Panadería Salvador

## 1. Carga inicial de datos

En este primer notebook del proyecto, vamos a realizar la **carga de datos brutos** de la Panadería Salvador. El objetivo es importar los archivos originales (ventas, productos, clientes, etc.), realizar una primera inspección básica y dejar los datos listos para el análisis exploratorio y el modelado posterior.

**Pasos de este notebook:**
- Descripción breve de los datos disponibles.
- Carga de los archivos originales al entorno de trabajo.
- Comprobación inicial de la integridad y estructura de los datos.

---

> **Nota:** Todos los análisis, visualizaciones y modelos posteriores del proyecto partirán de esta carga inicial.


### ¿Por qué usamos SQLAlchemy y Polars?

- **SQLAlchemy**: Permite conectar fácilmente a varias bases de datos y es más flexible y profesional que `mysql-connector-python`.
- **Polars**: Es mucho más rápido y eficiente que pandas para el análisis y procesamiento de grandes volúmenes de datos.


In [1]:
from sqlalchemy import create_engine, text  # Función "text" para ejecutar consultas SQL
import polars as pl  # He decidido probar Polars, por si es más rápido que Pandas
from dotenv import load_dotenv  # Carga variables desde el archivo .env
import os    # Módulo de Python para interactuar fácilmente con carpetas y archivos

load_dotenv()


# Parámetros de conexión a la base de datos
user = os.getenv("DB_USER")
host= os.getenv("DB_HOST")
port = 3306  # Puerto por defecto de MySQL
password= os.getenv("DB_PASSWORD")
nombre_base_datos = os.getenv("DB_NAME")   

# Crear el engine de SQLAlchemy
engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{nombre_base_datos}"
)

# Comprobar si la conexión es válida
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1;"))
        print("Conexión exitosa", result.fetchone())
except Exception as e:
    print("Error en la conexión:", e)

Conexión exitosa (1,)


In [2]:
import os  

DATA_RAW_DIR = r"d:\PersonalProjects\Panadería Datathon\data\raw"

# He usado fastexcel para leer archivos excel sin usar Pandas (por probar)
df_ventas = pl.read_excel(os.path.join(DATA_RAW_DIR, "ArticulosPanaderia.xlsx"))
df_calendario = pl.read_excel(os.path.join(DATA_RAW_DIR, "Calendario.xlsx"))
df_pedidos = pl.read_excel(os.path.join(DATA_RAW_DIR, "CantidadPedida.xlsx"))

In [3]:
from sqlalchemy import create_engine, Table, MetaData, Column, Integer, String, Float, Date

metadata = MetaData()

# Tabla de ventas
rem_ventas = Table(
    "rem_ventas", metadata,
    Column("FAMILIA", String(255)),
    Column("Tipo", String(255)),
    Column("FechaVenta", Date),
    Column("HoraVenta", Integer),
    Column("Articulo", String(255)),
    Column("Cantidad", Float),
    Column("Precio", Float),
    Column("Importe", Float),
)

# Tabla de calendario
rem_calendario = Table(
    "rem_calendario", metadata,
    Column("Fecha", Date),
    Column("Festivo", String(255)),
)

# Tabla de pedidos
rem_pedidos = Table(
    "rem_pedidos", metadata,
    Column("Familia", String(255)),
    Column("Tipo", String(255)),
    Column("Fecha", Date),
    Column("Articulo", String(255)),
    Column("Cantidad", Float),
    Column("Precio", Float),
    Column("Importe", Float),
)

metadata.create_all(engine)


In [4]:
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import insert

# Inserción por lotes, ya que da error de conexión la tabla ventas
CHUNK_SIZE = 1000 # Muestra de 1000 registros

def insertar_chunks(conn, tabla, df, nombre_tabla, chunk_size=CHUNK_SIZE):
    registros = df.to_dicts() # Cada registro es un dict, ya que SQLAlchemy los mapea
    total = len(registros)
    for i in range(0, total, chunk_size):
        lote = registros[i:i+chunk_size]
        try:
            conn.execute(insert(tabla), lote)
            print(f"Chunk {i//chunk_size+1} de {nombre_tabla} ({len(lote)} filas) insertado")
        except SQLAlchemyError as e:
            print(f"Error en chunk {i//chunk_size+1} de {nombre_tabla}: {e}")
            raise    # Aquí detiene la ejecución y muestra el error

with engine.begin() as conn:
    insertar_chunks(conn, rem_ventas, df_ventas, "rem_ventas")
    insertar_chunks(conn, rem_calendario, df_calendario, "rem_calendario")
    insertar_chunks(conn, rem_pedidos, df_pedidos, "rem_pedidos")
    

Chunk 1 de rem_ventas (1000 filas) insertado
Chunk 2 de rem_ventas (1000 filas) insertado
Chunk 3 de rem_ventas (1000 filas) insertado
Chunk 4 de rem_ventas (1000 filas) insertado
Chunk 5 de rem_ventas (1000 filas) insertado
Chunk 6 de rem_ventas (1000 filas) insertado
Chunk 7 de rem_ventas (1000 filas) insertado
Chunk 8 de rem_ventas (1000 filas) insertado
Chunk 9 de rem_ventas (1000 filas) insertado
Chunk 10 de rem_ventas (1000 filas) insertado
Chunk 11 de rem_ventas (1000 filas) insertado
Chunk 12 de rem_ventas (1000 filas) insertado
Chunk 13 de rem_ventas (1000 filas) insertado
Chunk 14 de rem_ventas (1000 filas) insertado
Chunk 15 de rem_ventas (1000 filas) insertado
Chunk 16 de rem_ventas (1000 filas) insertado
Chunk 17 de rem_ventas (1000 filas) insertado
Chunk 18 de rem_ventas (1000 filas) insertado
Chunk 19 de rem_ventas (1000 filas) insertado
Chunk 20 de rem_ventas (1000 filas) insertado
Chunk 21 de rem_ventas (1000 filas) insertado
Chunk 22 de rem_ventas (1000 filas) inserta

### Inserción eficiente de datos: aprendiendo y experimentando

En este  proyecto, he probado distintas formas de cargar grandes volúmenes de datos (como la tabla de ventas) en MySQL.  
Finalmente, me he decantado por la inserción en lotes (*chunks*) usando Polars y SQLAlchemy, por varias razones:

- **Simplicidad y limpieza**: El código es mucho más corto y claro, sin necesidad de recorrer filas una a una ni construir registros manualmente.
- **Rendimiento**: La inserción por lotes evita errores de conexión y permite que la carga sea sorprendentemente rápida (la tabla ventas, que es la mayor, se ha cargado en solo 1 minuto y 40 segundos).
- **Aprendizaje**: Creo que he conseguido mejorar el rendimiento al notebook original, trabajando de forma más eficiente con grandes conjuntos de datos y bases SQL en Python, experimentando con distintos tamaños de lote (*chunk size* o muestras más pequeñas).

En resumen, experimentar con Polars y SQLAlchemy, junto con la inserción en *chunks*, me ha permitido cargar los datos de manera profesional, rápida y con un código más fácil de mantener.


#### Nota: las consultas siguientes están preparadas para lanzarse con los datos cargados en la bbdd data y con los nombres iniciales, si se lanzan con usuario1 debe adaptarse

#### Voy a intentar hacer las consultas con SQLAlchemy


In [5]:
# Dejo esto aquí, porqué he tenido problemas de conexión con el servidor, con esta celda compruebo si es error de SQL o de conexión
from sqlalchemy import text
with engine.connect() as conn:
    result = conn.execute(text("SELECT 1;"))   # Una simple consulta para ver si funciona
    print(result.fetchone())


(1,)


In [6]:
# En vez de una sola consulta, voy a dividirlas en sentencias, aquí SQLAlchemy no es tan eficiente, detecta errores al hacer la consulta grande
from sqlalchemy import text

sentencia = """
drop table if exists sandbox.articulos_top;
create table sandbox.articulos_top as (
    select
        Articulo,
        FAMILIA,
        sum(importe) as importe_total,
        ROW_NUMBER () OVER(partition by FAMILIA order by sum(importe) desc) as orden
    from sandbox.rem_ventas
    where FechaVenta >= '2021-05-01'
    group by 1,2
);
"""

with engine.connect() as conn:
    for stmt in sentencia.strip().split(';'):
        if stmt.strip():
            try:
                conn.execute(text(stmt + ';'))
                print("Sentencia ejecutada correctamente:\n", stmt[:100], "...")
            except Exception as e:
                print("Error ejecutando la sentencia:\n", stmt[:100], "...\n", e)
                raise


Sentencia ejecutada correctamente:
 drop table if exists sandbox.articulos_top ...
Sentencia ejecutada correctamente:
 
create table sandbox.articulos_top as (
    select
        Articulo,
        FAMILIA,
        sum(i ...


In [7]:
from sqlalchemy import text

sentencia_drop = "DROP TABLE IF EXISTS sandbox.calendario_dias;"

with engine.connect() as conn:
    try:
        conn.execute(text(sentencia_drop))
        print("DROP ejecutado correctamente")
    except Exception as e:
        print("Error ejecutando el DROP:\n", e)
        raise


DROP ejecutado correctamente


In [8]:
# Me ha dado errores al realizar la siguiente sentencia, he descubierto que MySQL corta las CTEs recursivas tras 1000 iteraciones

# Así que he tenido que aumentar el límite de iteraciones para los dias que hay entre 2017 y 2023

with engine.connect() as conn:
    conn.execute(text("SET SESSION cte_max_recursion_depth = 4000;"))
    print("Límite de recursión aumentado")

Límite de recursión aumentado


In [9]:
# Esta sentencia es la que fallaba constantemente al iterar más de 1000, en este caso días entre 2017 y 2023
from sqlalchemy import text

sentencia_create = """
CREATE TABLE sandbox.calendario_dias AS
WITH RECURSIVE cte_calendario AS (
    SELECT DATE('2017-01-01') AS calendar_date
    UNION ALL
    SELECT DATE_ADD(calendar_date, INTERVAL 1 DAY) AS calendar_date FROM cte_calendario
    WHERE DATE_ADD(calendar_date, INTERVAL 1 DAY) <= DATE('2023-12-31')
)
SELECT
    calendar_date AS fecha,
    YEAR(calendar_date) AS fx_anno,
    MONTH(calendar_date) AS fx_mes,
    DAY(calendar_date) AS fx_day,
    DATE_FORMAT(calendar_date, '%Y%m') AS fx_anno_mes,
    DATE_FORMAT(calendar_Date, '%x-%v') AS semana
FROM cte_calendario;
"""

print(sentencia_create)  
with engine.connect() as conn:
    try:
        conn.execute(text(sentencia_create))
        print("CREATE ejecutado correctamente")
    except Exception as e:
        print("Error ejecutando el CREATE:\n", e)
        raise



CREATE TABLE sandbox.calendario_dias AS
WITH RECURSIVE cte_calendario AS (
    SELECT DATE('2017-01-01') AS calendar_date
    UNION ALL
    SELECT DATE_ADD(calendar_date, INTERVAL 1 DAY) AS calendar_date FROM cte_calendario
    WHERE DATE_ADD(calendar_date, INTERVAL 1 DAY) <= DATE('2023-12-31')
)
SELECT
    calendar_date AS fecha,
    YEAR(calendar_date) AS fx_anno,
    MONTH(calendar_date) AS fx_mes,
    DAY(calendar_date) AS fx_day,
    DATE_FORMAT(calendar_date, '%Y%m') AS fx_anno_mes,
    DATE_FORMAT(calendar_Date, '%x-%v') AS semana
FROM cte_calendario;

CREATE ejecutado correctamente


In [10]:
from sqlalchemy import text

# DROP
with engine.connect() as conn:
    try:
        conn.execute(text("DROP TABLE IF EXISTS sandbox.calendario_completo;"))
        print("DROP calendario_completo ejecutado correctamente")
    except Exception as e:
        print("Error en el DROP:\n", e)
        raise




DROP calendario_completo ejecutado correctamente


In [11]:
# CREATE
sentencia_create = """
CREATE TABLE sandbox.calendario_completo AS
SELECT
    base.*,
    festivos.festivo
FROM sandbox.calendario_dias base
LEFT JOIN (
    SELECT
        a.*,
        ROW_NUMBER() OVER(PARTITION BY a.fecha ORDER BY a.festivo) AS orden
    FROM sandbox.rem_calendario a
) festivos
ON base.fecha = festivos.fecha
AND festivos.orden = 1;
"""

with engine.connect() as conn:
    try:
        conn.execute(text(sentencia_create))
        print("CREATE calendario_completo ejecutado correctamente")
    except Exception as e:
        print("Error ejecutando el CREATE:\n", e)
        raise

CREATE calendario_completo ejecutado correctamente


In [12]:
# DROP
with engine.connect() as conn:
    try:
        conn.execute(text("DROP TABLE IF EXISTS sandbox.ventas_diarias;"))
        print("DROP ventas_diarias ejecutado correctamente")
    except Exception as e:
        print("Error en el DROP:\n", e)
        raise

DROP ventas_diarias ejecutado correctamente


In [13]:
# CREATE
sentencia_create = """
CREATE TABLE sandbox.ventas_diarias AS
SELECT
    base.familia,
    base.tipo,
    base.fechaVenta,
    calendario.festivo,
    base.articulo,
    SUM(base.precio * base.cantidad) / SUM(base.cantidad) AS precio,
    articulos.orden AS orden_articulo_familia,
    CASE WHEN base.fechaVenta >= DATE('2021-05-01') THEN 'S' ELSE 'N' END AS in_fecha_estudio,
    SUM(base.cantidad) AS cantidad,
    SUM(base.importe) AS importe
FROM sandbox.rem_ventas base
INNER JOIN sandbox.calendario_completo calendario
    ON base.FechaVenta = calendario.fecha
INNER JOIN sandbox.articulos_top articulos
    ON base.familia = articulos.familia
    AND base.articulo = articulos.articulo
GROUP BY 1,2,3,4,5,7,8;
"""

with engine.connect() as conn:
    try:
        conn.execute(text(sentencia_create))
        print("CREATE ventas_diarias ejecutado correctamente")
    except Exception as e:
        print("Error ejecutando el CREATE:\n", e)
        raise

CREATE ventas_diarias ejecutado correctamente


In [14]:
# DROP
with engine.connect() as conn:
    try:
        conn.execute(text("DROP VIEW IF EXISTS sandbox.ventas_diarias_estudio_completo;"))
        print("DROP vista ventas_diarias_estudio_completo ejecutado correctamente")
    except Exception as e:
        print("Error en el DROP:\n", e)
        raise

DROP vista ventas_diarias_estudio_completo ejecutado correctamente


In [15]:
# CREATE
sentencia_create = """
CREATE VIEW sandbox.ventas_diarias_estudio_completo AS
SELECT *
FROM sandbox.ventas_diarias
WHERE tipo = 'VENTA'
  AND in_fecha_estudio = 'S'
  AND orden_articulo_familia <= 5;
"""

with engine.connect() as conn:
    try:
        conn.execute(text(sentencia_create))
        print("CREATE vista ventas_diarias_estudio_completo ejecutado correctamente")
    except Exception as e:
        print("Error ejecutando el CREATE VIEW:\n", e)
        raise

CREATE vista ventas_diarias_estudio_completo ejecutado correctamente


In [16]:
# DROP
with engine.connect() as conn:
    try:
        conn.execute(text("DROP VIEW IF EXISTS sandbox.ventas_diarias_estudio;"))
        print("DROP vista ventas_diarias_estudio ejecutado correctamente")
    except Exception as e:
        print("Error en el DROP:\n", e)
        raise

DROP vista ventas_diarias_estudio ejecutado correctamente


In [17]:
# CREATE
sentencia_create = """
CREATE VIEW sandbox.ventas_diarias_estudio AS
SELECT *
FROM sandbox.ventas_diarias
WHERE tipo = 'VENTA'
  AND in_fecha_estudio = 'S'
  AND fechaventa < DATE('2023-05-01')
  AND orden_articulo_familia <= 5;
"""

with engine.connect() as conn:
    try:
        conn.execute(text(sentencia_create))
        print("CREATE vista ventas_diarias_estudio ejecutado correctamente")
    except Exception as e:
        print("Error ejecutando el CREATE VIEW:\n", e)
        raise

CREATE vista ventas_diarias_estudio ejecutado correctamente


In [18]:
# Pequeña comprobación para ver que las nuevas tablas existen y contienen datos

with engine.connect() as conn:
    result = conn.exec_driver_sql("SELECT COUNT(*) FROM sandbox.ventas_diarias;")
    print("Filas en ventas_diarias:", result.fetchone()[0])

Filas en ventas_diarias: 177589


In [19]:
# Comprobación que las tablas estan correctas y recuento de filas
from sqlalchemy import text

tablas_a_comprobar = [
    "sandbox.articulos_top",
    "sandbox.calendario_dias",
    "sandbox.calendario_completo",
    "sandbox.ventas_diarias"
]

with engine.connect() as conn:
    for tabla in tablas_a_comprobar:
        try:
            result = conn.execute(text(f"SELECT COUNT(*) FROM {tabla};"))
            n_filas = result.scalar()
            print(f"✔ {tabla}: {n_filas} filas")
        except Exception as e:
            print(f"❌ Error comprobando {tabla}:", e)


✔ sandbox.articulos_top: 170 filas
✔ sandbox.calendario_dias: 2556 filas
✔ sandbox.calendario_completo: 2556 filas
✔ sandbox.ventas_diarias: 177589 filas


In [20]:
# Comprobar vistas y primeras filas

vistas_a_comprobar = [
    "sandbox.ventas_diarias_estudio_completo",
    "sandbox.ventas_diarias_estudio"
]

with engine.connect() as conn:
    for vista in vistas_a_comprobar:
        try:
            result = conn.execute(text(f"SELECT * FROM {vista} LIMIT 5;"))
            filas = result.fetchall()
            print(f"\n✔ {vista} (primeras 5 filas):")
            for fila in filas:
                print(fila)
        except Exception as e:
            print(f"❌ Error comprobando {vista}:", e)



✔ sandbox.ventas_diarias_estudio_completo (primeras 5 filas):
('BOLLERIA', 'VENTA', datetime.date(2021, 5, 1), None, '3880', 2.5910000801086426, 3, 'S', 1995.0, 5169.044982910156)
('BOLLERIA', 'VENTA', datetime.date(2021, 5, 1), None, '3960', 2.318000078201294, 1, 'S', 2814.0, 6522.851963043213)
('BOLLERIA', 'VENTA', datetime.date(2021, 5, 1), None, '5803', 2.7269999980926514, 5, 'S', 1659.0, 4524.092864990234)
('BOLLERIA', 'VENTA', datetime.date(2021, 5, 1), None, '6286', 3.135999917984009, 4, 'S', 1155.0, 3622.0800437927246)
('BOLLERIA', 'VENTA', datetime.date(2021, 5, 1), None, '6425', 31.364000320434567, 2, 'S', 222.0749968290329, 6965.154582977295)

✔ sandbox.ventas_diarias_estudio (primeras 5 filas):
('BOLLERIA', 'VENTA', datetime.date(2021, 5, 1), None, '3880', 2.5910000801086426, 3, 'S', 1995.0, 5169.044982910156)
('BOLLERIA', 'VENTA', datetime.date(2021, 5, 1), None, '3960', 2.318000078201294, 1, 'S', 2814.0, 6522.851963043213)
('BOLLERIA', 'VENTA', datetime.date(2021, 5, 1),

In [21]:
# Verificar nombres de columnas
from sqlalchemy import text

tabla = "sandbox.ventas_diarias"
with engine.connect() as conn:
    result = conn.execute(text(f"SHOW COLUMNS FROM {tabla};"))
    columnas = [row[0] for row in result.fetchall()]
    print(f"Columnas en {tabla}:", columnas)


Columnas en sandbox.ventas_diarias: ['familia', 'tipo', 'fechaVenta', 'festivo', 'articulo', 'precio', 'orden_articulo_familia', 'in_fecha_estudio', 'cantidad', 'importe']


## Nota sobre la ejecución de consultas SQL y el uso de SQLAlchemy

Durante la preparación y carga de datos, he tenido que **dividir las consultas SQL en varias ejecuciones individuales** en vez de ejecutar todo el bloque de una vez. Esto se debe a que el driver `pymysql` de SQLAlchemy (que conecta con MySQL) no permite ejecutar varias sentencias (por ejemplo, DROP y CREATE) juntas en una sola llamada. Además, algunas sentencias complejas requieren ejecutarse de forma separada para evitar errores de sintaxis y problemas con el formateo de cadenas (especialmente cuando se usan funciones como `date_format` con `%`).

**Sobre el uso de SQLAlchemy:**  
He optado por usar SQLAlchemy por su flexibilidad y porque permite una gestión más profesional y portable de la conexión y las transacciones a la base de datos. Es cierto que, en algunos casos, el conector `mysql-connector-python` podría haber simplificado la ejecución de bloques largos de SQL, pero trabajar con SQLAlchemy me ha permitido aprender y aplicar buenas prácticas de desarrollo en proyectos de análisis de datos con Python.

En resumen, aunque ejecutar las consultas una a una puede parecer menos eficiente al principio, garantiza que cada paso del proceso se controla y documenta correctamente, y me ha servido para entender mejor cómo interactúan Python y SQL en proyectos reales.
